In [16]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="6"

In [32]:

import numpy as np
from matplotlib import pyplot as plt
import nibabel as nib
from PIL import Image
from matplotlib import pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from sklearn.model_selection import train_test_split
import torchsummary
from torchvision import transforms
import time
from sklearn.metrics import classification_report

In [18]:
#파일이름 sort해서 list저장
data_path='/disk1/data_liverbound_noclip/'
name_list=os.listdir(data_path)

segmentation_data = [files[:-4] for files in name_list if files.startswith('segmentation')]
segmentation_data=list(set(segmentation_data))
segmentation_data.sort()

volume_data=[files[:-4] for files in name_list if files.startswith('volume')]
volume_data=list(set(volume_data))
volume_data.sort()

In [19]:
all_labels=np.loadtxt('/home/sumins/workspace/liver_classification/all_labels.txt',dtype=int)
all_labels=all_labels.tolist()

In [20]:
# #npy를 slice별로 나누어 하나의 list저장
# segmentation_list=[]
# for file in segmentation_data:
#     fname=os.path.basename(file)
#     print(fname)
#     img_array=np.load(data_path+fname+'.npy')
    
#     #print(f'img_num: {img_array.shape}')
    
#     if len(img_array.shape) == 3:
#         nx, ny, nz = img_array.shape
#         total_slices = img_array.shape[2]
#         print(total_slices)
#         # iterate through slices
#         for current_slice in range(0, total_slices):
#             segmentation_list.append(img_array[:,:,current_slice]) 
# #간 1, 병변 2, 나머지 0


volume_list=[]
for file in volume_data:
    fname=os.path.basename(file)
    print(fname)
    img_array=np.load(data_path+fname+'.npy')
    
    #print(f'img_num: {img_array.shape}')
    
    if len(img_array.shape) == 3:
        nx, ny, nz = img_array.shape
        total_slices = img_array.shape[2]
        # iterate through slices
        for current_slice in range(0, total_slices):
            volume_list.append(img_array[:,:,current_slice]) 

volume-0
volume-1
volume-10
volume-100
volume-101
volume-102
volume-103
volume-104
volume-105
volume-106
volume-107
volume-108
volume-109
volume-11
volume-110
volume-111
volume-112
volume-113
volume-114
volume-115
volume-116
volume-117
volume-118
volume-119
volume-12
volume-120
volume-121
volume-122
volume-123
volume-124
volume-125
volume-126
volume-127
volume-128
volume-129
volume-13
volume-14
volume-15
volume-16
volume-17
volume-18
volume-19
volume-2
volume-20
volume-21
volume-22
volume-23
volume-24
volume-25
volume-26
volume-27
volume-28
volume-29
volume-3
volume-30
volume-31
volume-32
volume-33
volume-34
volume-35
volume-36
volume-37
volume-38
volume-39
volume-4
volume-40
volume-41
volume-42
volume-43
volume-44
volume-45
volume-46
volume-47
volume-48
volume-49
volume-5
volume-50
volume-51
volume-52
volume-53
volume-54
volume-55
volume-56
volume-57
volume-58
volume-59
volume-6
volume-60
volume-61
volume-62
volume-63
volume-64
volume-65
volume-66
volume-67
volume-68
volume-69
volume-

In [29]:
# #label을 만들어 list에 저장
# all_labels = []
# for i in segmentation_list:
#     if 2 in i:
#         all_labels.append(1)
#     else:
#         all_labels.append(0)

In [21]:
def norm(nparray):
    # normalize scans to [0,1]
    _min = nparray.min()
    _max = nparray.max()
    nparray = nparray - _min
    nparray = nparray / (_max - _min)
    return nparray

def norm_zscore(nparray):
    # normalize 2d scands by mean and standard deviation
    mean = nparray.mean()
    std = nparray.std()    
    nparray = nparray - mean
    nparray /= std
    return nparray

In [22]:
WINDOW_MAX = 200
WINDOW_MIN = 0
GLOBAL_PIXEL_MEAN = 0.1

class CustomDataset(Dataset): 
  def __init__(self,volume_list,all_labels,transforms=None):
    self.volume_list=volume_list
    self.all_labels=all_labels
    self.length=len(all_labels)
    self.transforms=transforms


  def __len__(self):
    return self.length


  def __getitem__(self, idx):
    npy=self.volume_list[idx]

    npy[npy > WINDOW_MAX] = WINDOW_MAX
    npy[npy < WINDOW_MIN] = WINDOW_MIN
    
    npy = (npy - WINDOW_MIN) / (WINDOW_MAX - WINDOW_MIN)
    npy -= GLOBAL_PIXEL_MEAN
    
    if len(npy.shape)==2:
      npy=npy[:,:,np.newaxis].astype(dtype='float32')
    
    if self.transforms is not None:
      npy=self.transforms(npy)
    
    return{'npy':npy,'label':self.all_labels[idx]}

In [23]:
class CustomConvNet(nn.Module):
    def __init__(self):
        super(CustomConvNet, self).__init__() #상속받은 class에 접근하기 위함

        self.layer1 = self.conv_module(1, 16) #흑백사진은 inp
        self.layer2 = self.conv_module(16, 32)
        self.layer3 = self.conv_module(32, 64)
        self.layer4 = self.conv_module(64, 128)
        self.layer5 = self.conv_module(128, 256)
        self.gap = self.global_avg_pool(256, 2)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.gap(out)
        out = out.view(-1, 2)

        return out

    def conv_module(self, in_num, out_num):
        return nn.Sequential(
            nn.Conv2d(in_num, out_num, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_num),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

    def global_avg_pool(self, in_num, out_num):
        return nn.Sequential(
            nn.Conv2d(in_num, out_num, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_num),
            nn.LeakyReLU(),
            nn.AdaptiveAvgPool2d((1,1)))
        
model=CustomConvNet()
torchsummary.summary(model.cuda(),input_size=(1,512,512),batch_size=1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 16, 512, 512]             160
       BatchNorm2d-2          [1, 16, 512, 512]              32
         LeakyReLU-3          [1, 16, 512, 512]               0
         MaxPool2d-4          [1, 16, 256, 256]               0
            Conv2d-5          [1, 32, 256, 256]           4,640
       BatchNorm2d-6          [1, 32, 256, 256]              64
         LeakyReLU-7          [1, 32, 256, 256]               0
         MaxPool2d-8          [1, 32, 128, 128]               0
            Conv2d-9          [1, 64, 128, 128]          18,496
      BatchNorm2d-10          [1, 64, 128, 128]             128
        LeakyReLU-11          [1, 64, 128, 128]               0
        MaxPool2d-12            [1, 64, 64, 64]               0
           Conv2d-13           [1, 128, 64, 64]          73,856
      BatchNorm2d-14           [1, 128,

In [24]:
transforms_train = transforms.Compose([transforms.ToTensor()
                                       ])
transforms_test = transforms.Compose([transforms.ToTensor()
                                       ])

hyper_param_epoch=50
hyper_param_batch=32
hyper_param_learning_rate=0.0001

In [26]:
vol_train, vol_valid, lab_train, lab_valid = train_test_split(volume_list, all_labels, test_size=0.3, shuffle=True, stratify=all_labels, random_state=34)
train_dataset=CustomDataset(volume_list=vol_train, all_labels=lab_train,transforms=transforms_train)
test_dataset=CustomDataset(volume_list=vol_valid,all_labels=lab_valid,transforms=transforms_test)
train_loader = DataLoader(train_dataset, batch_size=hyper_param_batch, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=hyper_param_batch, shuffle=True)

In [27]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device,"/" ,torch.cuda.device_count())

custom_model=CustomConvNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(custom_model.parameters(), lr=hyper_param_learning_rate)

cuda:0 / 1


In [29]:
loss_value=1
start=time.time()
custom_model.train()
for e in range(hyper_param_epoch):
        for i_batch, item in enumerate(train_loader):
                npys = item['npy'].to(device)
                labels = item['label'].to(device)
                #print(npys)
                # Forward pass
                outputs =custom_model(npys)
                loss = criterion(outputs, labels)

                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
 

        print('Epoch [{}/{}],i_batch={} Loss: {:.4f}'
                                        .format(e + 1, hyper_param_epoch, i_batch+1, loss.item()))
        print("Time: {}sec".format(time.time()-start))
        start=time.time()
        if loss_value>loss.item():
                loss_value=loss.item()
                torch.save({
                'epoch': e,
                'model_state_dict': custom_model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
                }, '/home/sumins/workspace/model_check/model_best.pth')


Epoch [1/50],i_batch=486 Loss: 0.6047
Time: 53.23007273674011sec
Epoch [2/50],i_batch=486 Loss: 0.5139
Time: 53.55405402183533sec
Epoch [3/50],i_batch=486 Loss: 0.4745
Time: 53.7421178817749sec
Epoch [4/50],i_batch=486 Loss: 0.5293
Time: 53.87000298500061sec
Epoch [5/50],i_batch=486 Loss: 0.4741
Time: 53.942797899246216sec
Epoch [6/50],i_batch=486 Loss: 0.4325
Time: 53.95321702957153sec
Epoch [7/50],i_batch=486 Loss: 0.4161
Time: 53.94296622276306sec
Epoch [8/50],i_batch=486 Loss: 0.3957
Time: 53.91041135787964sec
Epoch [9/50],i_batch=486 Loss: 0.3790
Time: 54.00072169303894sec
Epoch [10/50],i_batch=486 Loss: 0.3516
Time: 53.94689989089966sec
Epoch [11/50],i_batch=486 Loss: 0.3445
Time: 53.923327684402466sec
Epoch [12/50],i_batch=486 Loss: 0.3407
Time: 53.91600036621094sec
Epoch [13/50],i_batch=486 Loss: 0.2793
Time: 53.92257785797119sec
Epoch [14/50],i_batch=486 Loss: 0.3464
Time: 53.90531635284424sec
Epoch [15/50],i_batch=486 Loss: 0.2958
Time: 53.920103311538696sec
Epoch [16/50],i_b

In [31]:
true_label=[]
pred_label=[]
custom_model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for item in test_loader:
        npys = item['npy'].to(device)
        labels = item['label'].to(device)
        
        outputs = custom_model(npys)
        _, predicted = torch.max(outputs.data, 1)
        true_label.extend(labels)
        pred_label.extend(predicted)
        total += len(labels)
        correct += (predicted == labels).sum().item()
        
    print('Test Accuracy of the model on the {} test images: {} %'.format(total, 100 * correct / total))

Test Accuracy of the model on the 6663 test images: 92.9611286207414 %


In [37]:
true_labels=torch.tensor(true_label)
true_labels=true_labels.tolist()
pred_labels=torch.tensor(pred_label)
pred_labels=pred_labels.tolist()
print(classification_report(true_labels,pred_labels))

              precision    recall  f1-score   support

           0       1.00      0.90      0.95      4566
           1       0.82      1.00      0.90      2097

    accuracy                           0.93      6663
   macro avg       0.91      0.95      0.92      6663
weighted avg       0.94      0.93      0.93      6663



In [38]:

model=CustomConvNet().to(device)
optimizer = torch.optim.Adam(custom_model.parameters(), lr=hyper_param_learning_rate)

checkpoint = torch.load('/home/sumins/workspace/model_check/model_best.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
print(loss)
model.eval()
# with torch.no_grad():
#     correct = 0
#     total = 0
#     for item in test_loader:
#         npys = item['npy'].to(device)
#         labels = item['label'].to(device)
        
#         outputs = model(npys)
#         _, predicted = torch.max(outputs.data, 1)
#         total += len(labels)
#         correct += (predicted == labels).sum().item()
        
#     print('Test Accuracy of the model on the {} test images: {} %'.format(total, 100 * correct / total))



tensor(0.0459, device='cuda:0', requires_grad=True)


CustomConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer4): Sequential(
    (0): Conv2

: 